<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import DistilBertForSequenceClassification, DistilBertConfig
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
# from transformers import AutoTokenizer

from captum.attr import (
    visualization
)
import torch
from conceptshap import *
import numpy as np
from BERT_explainability.modules.layers_ours import *
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
import os
from BERT_explainability.modules.layers_ours import NormLayer

## Load BERT model

In [3]:
# # model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2").to("cuda")
# save_dir = 'models/imdb/bert/imdb_weights'
# # model = DistilBertForSequenceClassification.from_pretrained(save_dir)
# model = BertForSequenceClassification.from_pretrained(save_dir).to(device)
# model.eval()
# # tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [4]:
from transformers import BertConfig
config = BertConfig.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification(config)

In [25]:
t = torch.rand(1, 512, 768)
t.flatten(start_dim = 0, end_dim = 1).shape

torch.Size([512, 768])

In [10]:
list(model.children())
# bertmodel, dropout, linear

model.classifier
# linear

Linear(in_features=768, out_features=2, bias=True)

In [23]:
list(model.bert.encoder.layer)

[BertLayer(
   (attention): BertAttention(
     (self): BertSelfAttention(
       (query): Linear(in_features=768, out_features=768, bias=True)
       (key): Linear(in_features=768, out_features=768, bias=True)
       (value): Linear(in_features=768, out_features=768, bias=True)
       (dropout): Dropout(p=0.1, inplace=False)
       (matmul1): MatMul()
       (matmul2): MatMul()
       (softmax): Softmax(dim=-1)
       (add): Add()
       (mul): Mul()
       (clone): Clone()
     )
     (output): BertSelfOutput(
       (dense): Linear(in_features=768, out_features=768, bias=True)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
       (add): Add()
     )
     (clone): Clone()
   )
   (intermediate): BertIntermediate(
     (dense): Linear(in_features=768, out_features=3072, bias=True)
     (intermediate_act_fn): GELU()
   )
   (output): BertOutput(
     (dense): Linear(in_features=3072, out_features=768, bias=True)

In [27]:
len(list(model.bert.children()))

3

In [34]:
model.dropout

Dropout(p=0.1, inplace=False)

In [32]:
list(model.children())[-1]

Linear(in_features=768, out_features=2, bias=True)

In [20]:
list(model.bert.children())
# embeddings, encoder, pooler

model.bert.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (matmul1): MatMul()
        (matmul2): MatMul()
        (softmax): Softmax(dim=-1)
        (add): Add()
        (mul): Mul()
        (clone): Clone()
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (add): Add()
      )
      (clone): Clone()
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELU()
    )
    (output): BertOutput(
      (dense): Linear(i

## Load Topic Model

In [4]:
# import topic model
classifier = model.classifier
f_train = torch.from_numpy(np.load('models/imdb/bert/train_embeddings.npy'))
n_concept = 10
thres = 0.2

# os.chdir('src/')
# topic_model = topic_model_main(classifier, f_train, n_concept, thres, device, bert = True)
graph_save_folder = 'models/imdb/bert/two_stage/'
topic_model = torch.load(graph_save_folder + 'topic_model_two_stage.pkl')

## 1. Visualize class

In [5]:
#layers
# topic_vector = topic_model.topic_vector.to(device)
linearlayer1 = Linear(768, n_concept, bias = False)
topic_vector_n = F.normalize(topic_model.topic_vector, dim = 0, p=2).transpose(1,0)
linearlayer1.weight = nn.Parameter(topic_vector_n)

thres = topic_model.thres
print('thres: ', thres)

linearlayer2 = Linear(n_concept, n_concept, bias = True)
linearlayer2.weight = nn.Parameter(torch.eye(n_concept), requires_grad=False)
linearlayer2.bias = nn.Parameter(torch.ones(n_concept)*(-thres), requires_grad = False)

linearlayer3 = Linear(topic_model.rec_vector_1.shape[0], topic_model.rec_vector_1.shape[1], bias = False)
linearlayer3.weight = nn.Parameter(topic_model.rec_vector_1.transpose(1,0))

linearlayer4 = Linear(topic_model.rec_vector_2.shape[0], topic_model.rec_vector_2.shape[1], bias = False)
linearlayer4.weight = nn.Parameter(topic_model.rec_vector_2.transpose(1,0))
# new_classifier = Sequential(linearlayer1)
# new classifier
new_classifier = Sequential(NormLayer(),
                           linearlayer1,
                            linearlayer2,
                            ReLU(),
                           NormLayer(),
                           linearlayer3,
                            ReLU(),
                           linearlayer4,
                           classifier).to(device)
model.classifier = new_classifier.to(device)

thres:  0.3


In [6]:
# initialize the explanations generator
explanations = Generator(model)

# classifications = ["NEGATIVE", "POSITIVE"]
classifications = range(n_concept)

In [7]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# print('1: ', expl)
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# print('2: ', expl)

# get the model classification
output = model(input_ids=input_ids, attention_mask=attention_mask)[0]
print('output: ', output)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
fig = visualization.visualize_text(vis_data_records)
with open("data.html", "w") as file:
    file.write(fig.data)

output:  tensor([[ 96.3010, -79.8617]], device='cuda:0', grad_fn=<AddmmBackward>)
[('[CLS]', 0.0), ('this', 0.7241052985191345), ('movie', 0.878639817237854), ('was', 0.8026872277259827), ('the', 0.33629634976387024), ('best', 0.41377565264701843), ('movie', 0.35376426577568054), ('i', 0.1672552227973938), ('have', 0.2121099829673767), ('ever', 0.08996672183275223), ('seen', 0.3126651346683502), ('!', 0.7803391218185425), ('some', 0.19109849631786346), ('scenes', 0.30642256140708923), ('were', 0.24444608390331268), ('ridiculous', 0.09867827594280243), (',', 0.028233131393790245), ('but', 0.215656116604805), ('acting', 0.7725077867507935), ('was', 1.0), ('great', 0.907504677772522), ('.', 0.1846427321434021), ('[SEP]', 0.20668818056583405)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (96.30),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


## 2. Visualize concept

In [8]:
#cut off classifier
new_cut_classifier = Sequential(NormLayer(),
                           linearlayer1,
                            linearlayer2,
                            ReLU(),
                           NormLayer()).to(device)
model.classifier = new_cut_classifier

In [9]:
# initialize the explanations generator
explanations = Generator(model)

# classifications = ["NEGATIVE", "POSITIVE"]
classifications = range(n_concept)

In [20]:
# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0, index = 9)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# print('2: ', expl)

# get the model classification
output = model(input_ids=input_ids, attention_mask=attention_mask)[0]
print('output: ', output)
indices = np.argwhere(output.cpu().detach().numpy()>0)
print('Classes with importance >0: ', [i[1] for i in indices])
classification = output.argmax(dim=-1).item()
print('classification: ', classification)
# get class name
class_name = classifications[classification]

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class, #this changes the column 'true_label'
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

output:  tensor([[0.0000, 0.0000, 0.4932, 0.0000, 0.4920, 0.0000, 0.4431, 0.0000, 0.2685,
         0.4961]], device='cuda:0', grad_fn=<DivBackward0>)
Classes with importance >0:  [2, 4, 6, 8, 9]
classification:  9
[('[CLS]', 0.0), ('this', 0.6721650958061218), ('movie', 0.4240603744983673), ('was', 0.5798816084861755), ('the', 0.4014686048030853), ('best', 0.8619788885116577), ('movie', 0.31506285071372986), ('i', 0.3781983554363251), ('have', 0.15347115695476532), ('ever', 0.08573821932077408), ('seen', 0.11877365410327911), ('!', 0.8603025674819946), ('some', 0.11571356654167175), ('scenes', 0.19586196541786194), ('were', 0.2765940725803375), ('ridiculous', 0.20337171852588654), (',', 0.14511419832706451), ('but', 0.25527259707450867), ('acting', 0.818137526512146), ('was', 1.0), ('great', 0.8157693147659302), ('.', 0.6944469213485718), ('[SEP]', 0.12099483609199524)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,9 (0.50),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,9 (0.50),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


**Negative sentiment example**

In [5]:
# encode a sentence
text_batch = ["I really didn't like this movie. Some of the actors were good, but overall the movie was boring."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.153622567653656), ('really', -0.16307400166988373), ('didn', -0.19001001119613647), ("'", -0.08315945416688919), ('t', -0.2679630219936371), ('like', -0.17981572449207306), ('this', -0.45753321051597595), ('movie', -0.1397773027420044), ('.', -0.22919468581676483), ('some', -0.08338094502687454), ('of', -0.03789211064577103), ('the', -0.05978512391448021), ('actors', -0.10391402989625931), ('were', -0.06579089164733887), ('good', -0.07998379319906235), (',', -0.06481628119945526), ('but', -0.3014720678329468), ('overall', -0.27188125252723694), ('the', -0.2926645576953888), ('movie', -0.13271962106227875), ('was', -0.6214057803153992), ('boring', -1.0), ('.', -0.31120413541793823), ('[SEP]', -0.06028567999601364)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.92),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.92),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"
